<a href="https://colab.research.google.com/github/mingmcs/cs598ccc/blob/main/cs598ccc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Azure open AI Service



## set up the env
* install the required packages
* create .env file,
  ```
  OPENAI_API_KEY="xxx"
  ```
* upload the file to /content folder
  * need to upload everytime when start a new session


In [1]:
!pip install --quiet openai python-dotenv
!git clone https://github.com/LogSummary/code-and-datasets.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 978.1 kB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
Cloning into 'code-and-datasets'...
remote: Enumerating objects: 185, done.
remote: Counting objects: 100% (185/185), done.
remote: Compressing objects: 100% (147/147), done.
remote: Total 185 (delta 37), reused 185 (delta 37), pack-reused 0
Receiving objects: 100% (185/185), 361.29 KiB | 5.56 MiB/s, done.
Resolving deltas: 100% (37/37), done.


In [2]:
# work around to wrap the text from response
from IPython.display import HTML, display

# autowrap the output
def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [3]:
import json

proxifier_log_fie = "/content/code-and-datasets/data/summary/logs/Proxifier.txt"

SUMMARY = 'summary'
LOG = 'log'
def get_record_type(key):
  if key.startswith("#summary:#"):
    return SUMMARY
  else:
    return LOG

def load_data(file):
  data_log = {}
  data_summary = []
  current_key = None
  lines = []
  record_type = LOG

  with open(file, 'r') as file:
    for line in file:
      line = line.strip()
      if not line:
        continue

      if line.startswith("#"):
        if current_key is not None:
          if get_record_type(current_key) == SUMMARY:
            data_summary.append(lines)
          else:
            data_log[current_key.replace("#", "")] = lines

        current_key = line
        lines = []
      else:
        lines.append(line)

    if current_key is not None:
      data_summary.append(lines)
  return data_log, data_summary

p_data, p_summary = load_data(proxifier_log_fie)
print(len(p_data))
print( ', '.join(p_data.keys()))
print('\n'.join(p_data['1']))
print(len(p_data['2']))
print(len(p_summary))

100
1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100
- proxy.cse.cuhk.edu.hk : 5070 open through proxy proxy.cse.cuhk.edu.hk : 5070 HTTPS
- proxy.cse.cuhk.edu.hk : 5070 open through proxy proxy.cse.cuhk.edu.hk : 5070 HTTPS
- proxy.cse.cuhk.edu.hk : 5070 open through proxy proxy.cse.cuhk.edu.hk : 5070 HTTPS
- proxy.cse.cuhk.edu.hk : 5070 open through proxy proxy.cse.cuhk.edu.hk : 5070 HTTPS
- proxy.cse.cuhk.edu.hk : 5070 open through proxy proxy.cse.cuhk.edu.hk : 5070 HTTPS
- proxy.cse.cuhk.edu.hk : 5070 close , 403 bytes sent , 426 bytes received , lifetime <1 sec
- proxy.cse.cuhk.edu.hk : 5070 open through proxy proxy.cse.cuhk.edu.hk : 5070 HTTPS
- 

In [5]:
import os
import openai
import dotenv
import pandas as pd
import time
dotenv.load_dotenv('./.env')

openai.api_type = "azure"
openai.api_version = "2023-09-15-preview"
openai.api_base = "https://aiops23.openai.azure.com/"
openai.api_key = os.getenv("OPENAI_API_KEY")

def open_ai_summary(engine, input):
  input = '\n'.join(input)
  # print(input)
  response = openai.ChatCompletion.create(
    engine=engine,
    messages=[
        {"role": "system", "content": "Summarize the given content with less than 20 words"},
        {"role": "user", "content": input}
    ],
    temperature=0.3,
    max_tokens=250,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None)

  print(response)

  return response['choices'][0]['message']['content']
  # return "input"

In [ ]:
engines = ['gpt3', 'gpt4']

results = pd.DataFrame(columns=['ref', 'gen3', 'gen4'])

for index, ref in enumerate(p_summary):
  # print(f"Key: {index}, Value: {input}")
  if index > 10:
    continue
  summarys = [ref]
  for engine in engines:
    summary = open_ai_summary(engine, p_data[str(index+1)])
    summarys.append(summary)
    time.sleep(10)

  new_row = pd.DataFrame([summarys], columns=results.columns)
  results = pd.concat([results, new_row], ignore_index=True)

results.to_csv('openai.csv', index=False)

In [8]:

zookeeper_log_fie = "/content/code-and-datasets/data/summary/logs/Zookeeper.txt"
engines = ['gpt3', 'gpt4']

results = pd.DataFrame(columns=['gen3', 'gen4'])

with open(zookeeper_log_fie, 'r') as file:
    input = file.read()
    summarys = []
    for engine in engines:
      summary = open_ai_summary(engine, input)
      summarys.append(summary)
      time.sleep(10)

    new_row = pd.DataFrame([summarys], columns=results.columns)
    results = pd.concat([results, new_row], ignore_index=True)
    results.to_csv('large_input.csv', index=False)

InvalidRequestError: ignored